In [ ]:
# PATH needs correct setup to use DeepDive
import os; PWD=os.getcwd(); HOME=os.environ["HOME"]; PATH=os.environ["PATH"]
# home directory installation
%env PATH=$HOME/local/bin:$PATH
# notebook-local installation
%env PATH=$PWD/deepdive/bin:$PATH

!type deepdive
no_deepdive_found = !type deepdive >/dev/null
if no_deepdive_found: # install it next to this notebook
    !bash -c 'PREFIX="$PWD"/deepdive bash <(curl -fsSL git.io/getdeepdive) deepdive_from_release'

env: PATH=/home/jovyan/local/bin:/opt/conda/bin:/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin
env: PATH=/ConfinedWater/deepdive-examples/spouse/deepdive/bin:/opt/conda/bin:/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin
deepdive is /usr/local/bin/deepdive


In [ ]:
# check if notebook kernel was launched in a Unicode locale
import locale; LC_CTYPE = locale.getpreferredencoding()
if LC_CTYPE != "UTF-8":
    raise EnvironmentError("Notebook is running in '%s' encoding not compatible with DeepDive's Unicode output.\n\nPlease restart notebook in a UTF-8 locale with a command like the following:\n\n    LC_ALL=en_US.UTF-8 jupyter notebook" % (LC_CTYPE))

In [ ]:
%%file app.ddlog
## Random variable to predict #################################################

# This application's goal is to predict whether a given pair of person mention
# are indicating a spouse relationship or not.
has_parenthood?(
    p1_id text,
    p2_id text
).

Overwriting app.ddlog


In [ ]:
!deepdive compile

2021-11-24 18:23:51.997671 ‘run/LATEST.COMPILE’ -> ‘20211124/182351.950657704’
2021-11-24 18:23:51.997761 ‘run/RUNNING.COMPILE’ -> ‘20211124/182351.950657704’
2021-11-24 18:23:51.997791 Parsing DeepDive application (/ConfinedWater/deepdive-examples/spouse) to generate:
2021-11-24 18:23:51.997819  run/compiled/deepdive.conf
2021-11-24 18:23:51.997845   from app.ddlog
2021-11-24 18:23:52.435137   from deepdive.conf
2021-11-24 18:23:52.435943  run/compiled/deepdive.conf.json
2021-11-24 18:23:52.618588 Performing sanity checks on run/compiled/deepdive.conf.json:
2021-11-24 18:23:52.639393  checking if input_extractors_well_defined
2021-11-24 18:23:52.639503  checking if input_schema_wellformed
2021-11-24 18:23:52.639760 Normalizing and adding built-in processes to the data flow to compile:
2021-11-24 18:23:52.640228  run/compiled/config-0.00-init_objects.json
2021-11-24 18:23:52.652549  run/compiled/config-0.01-parse_calibration.json
2021-11-24 18:23:52.665963  run/compiled/config-0.01-par

In [ ]:
!echo 'postgresql://'"${PGHOST:-localhost}"'/deepdive_parenthood_$USER' >db.url

In [ ]:
no_database_running = !deepdive db is_ready || echo $?
if no_database_running:
    PGDATA = "run/database/postgresql"
    !mkdir -p $PGDATA; test -s $PGDATA/PG_VERSION || pg_ctl init -D $PGDATA >/dev/null
    !nohup pg_ctl -D $PGDATA -l $PGDATA/logfile start >/dev/null

In [ ]:
!deepdive redo init/app

‘run/RUNNING’ -> ‘20211115/151734.338984787’
2021-11-15 15:17:34.548791 process/init/app/run.sh
‘run/FINISHED’ -> ‘20211115/151734.338984787’


In [ ]:
%%file -a app.ddlog

## Input Data #################################################################
articles(
    id      text,
    content text
).

Appending to app.ddlog


In [ ]:
!mkdir -p input

In [ ]:
%%file input/articles.tsj.sh
#!/usr/bin/env bash
set -euo pipefail
cd "$(dirname "$0")"

corpus=signalmedia/signalmedia-1m.jsonl
[[ -e "$corpus" ]] || {
    echo "ERROR: Missing $PWD/$corpus"
    echo "# Please Download it from http://research.signalmedia.co/newsir16/signal-dataset.html"
    echo
    echo "# Alternatively, use our sampled data by running:"
    echo "deepdive load articles input/articles-100.tsv.bz2"
    echo
    echo "# Or, skipping all NLP markup processes by running:"
    echo "deepdive create table sentences"
    echo "deepdive load sentences"
    echo "deepdive mark done sentences"
    false
} >&2

cat "$corpus" |
#grep -E 'wife|husband|married' |
#head -100 |
jq -r '[.id, .content] | map(@json) | join("\t")'

Overwriting input/articles.tsj.sh


In [ ]:
!chmod +x input/articles.tsj.sh

In [ ]:
!deepdive redo articles

‘run/RUNNING’ -> ‘20211115/151746.951013385’
2021-11-15 15:17:47.195395 process/init/relation/articles/run.sh
2021-11-15 15:17:47.195031 ################################################################################
2021-11-15 15:17:47.195102 # Host: 42cf3f8c4652
2021-11-15 15:17:47.195122 # DeepDive: deepdive v0.8.0-742-g4b812a1 (Linux x86_64)
2021-11-15 15:17:47.195137 export PATH='/usr/local/bin':"$PATH"
2021-11-15 15:17:47.195165 export DEEPDIVE_PWD='/ConfinedWater/deepdive-examples/spouse'
2021-11-15 15:17:47.195194 export DEEPDIVE_APP='/ConfinedWater/deepdive-examples/spouse'
2021-11-15 15:17:47.195223 cd "$DEEPDIVE_APP"/run
2021-11-15 15:17:47.195247 export DEEPDIVE_RUN_ID='20211115/151746.951013385'
2021-11-15 15:17:47.195272 # Plan: 20211115/151746.951013385/plan.sh
2021-11-15 15:17:47.195297 # Targets: articles
2021-11-15 15:17:47.195321 ################################################################################
2021-11-15 15:17:47.195346 
2021-11-15 15:17:47.195370   

In [ ]:
NUM_ARTICLES = 100
ARTICLES_FILE = "articles-%d.tsj.bz2" % NUM_ARTICLES

articles_not_done = !deepdive done articles || date
if articles_not_done:
    !cd input && curl -RLO "https://github.com/HazyResearch/deepdive/raw/master/examples/spouse/input/"$ARTICLES_FILE
    !deepdive reload articles input/$ARTICLES_FILE

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   171  100   171    0     0    913      0 --:--:-- --:--:-- --:--:--   914
100  135k  100  135k    0     0   325k      0 --:--:-- --:--:-- --:--:--  325k
CREATE TABLE
Loading articles from input/articles-100.tsj.bz2 (tsj format)
COPY 100
ANALYZE


In [ ]:
!deepdive query '|2 ?- articles(id, content).'

                  id                  |                                                                                                                                                                                                                                                                                                                                  content                                                                                                                                                                                                                                                                                                                                   
--------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
%%file -a app.ddlog

## NLP markup #################################################################
sentences(
    doc_id         text,
    sentence_index int,
    tokens         json,
    lemmas         json,
    pos_tags       json,
    ner_tags       json,
    doc_offsets    json,
    dep_types      json,
    dep_tokens     json
).

Appending to app.ddlog


In [ ]:
%%file -a app.ddlog

function nlp_markup over (
        doc_id  text,
        content text
    ) returns rows like sentences
    implementation "udf/nlp_markup.sh" handles tsj lines.

sentences += nlp_markup(doc_id, content) :-
    articles(doc_id, content).

Appending to app.ddlog


In [ ]:
%%file udf/nlp_markup.sh
#!/usr/bin/env bash
# Parse documents in tab-separated JSONs input stream with CoreNLP
#
# $ deepdive corenlp install
# $ deepdive corenlp start
# $ deepdive env udf/nlp_markup.sh
# $ deepdive corenlp stop
##
set -euo pipefail
cd "$(dirname "$0")"

# some configuration knobs for CoreNLP
: ${CORENLP_PORT:=$(deepdive corenlp unique-port)}  # a CoreNLP server started ahead of time is shared across parallel UDF processes
# See: http://stanfordnlp.github.io/CoreNLP/annotators.html
: ${CORENLP_ANNOTATORS:="
        tokenize
        ssplit
        pos
        ner
        lemma
        depparse
    "}
export CORENLP_PORT
export CORENLP_ANNOTATORS

# make sure CoreNLP server is available
deepdive corenlp is-running || {
    echo >&2 "PLEASE MAKE SURE YOU HAVE RUN: deepdive corenlp start"
    false
}

# parse input with CoreNLP and output a row for every sentence
deepdive corenlp parse-tsj docid+ content=nlp -- docid nlp |
deepdive corenlp sentences-tsj docid content:nlp \
                            -- docid nlp.{index,tokens.{word,lemma,pos,ner,characterOffsetBegin}} \
                                     nlp.collapsed-dependencies.{dep_type,dep_token}


Overwriting udf/nlp_markup.sh


In [ ]:
!chmod +x udf/nlp_markup.sh

In [ ]:
!deepdive corenlp install
# If CoreNLP seems to take forever to start, retry after uncommenting the following line:
%env CORENLP_JAVAOPTS=-Xmx4g
!deepdive corenlp start

CoreNLP already installed at /deepdive/lib/stanford-corenlp/corenlp
env: CORENLP_JAVAOPTS=-Xmx4g
CoreNLP server at CORENLP_PORT=24393 starting...
CoreNLP server at CORENLP_PORT=24393 ready.
To stop it after final use, run: deepdive corenlp stop
To watch its log, run: deepdive corenlp watch-log


In [ ]:
!deepdive redo sentences

‘run/RUNNING’ -> ‘20211115/152019.988902075’
2021-11-15 15:20:20.247910 process/ext_sentences_by_nlp_markup/run.sh
2021-11-15 15:20:43.837466 deepdive mark 'done' data/sentences
‘run/FINISHED’ -> ‘20211115/152019.988902075’


In [ ]:
%%bash
deepdive query '
    doc_id, index, tokens, ner_tags | 5
    ?- sentences(doc_id, index, tokens, lemmas, pos_tags, ner_tags, _, _, _).
'

                doc_id                | index |                                                                                                                                                                     tokens                                                                                                                                                                     |                                                                                                                                                         ner_tags                                                                                                                                                          
--------------------------------------+-------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
%%file -a app.ddlog

## Candidate mapping ##########################################################
person_mention1(
    mention_id     text,
    mention_text   text,
    doc_id         text,
    sentence_index int,
    begin_index    int,
    end_index      int,
    sentence       text  #added for easy ref
).


Appending to app.ddlog


In [ ]:
%%file -a app.ddlog

function map_person_mention1 over (
        doc_id         text,
        sentence_index int,
        tokens         text[],
        ner_tags       text[]
    ) returns rows like person_mention
    implementation "udf/map_person_mention.py" handles tsj lines.

person_mention1 += map_person_mention1(
    doc_id, sentence_index, tokens, ner_tags
) :-
    sentences(doc_id, sentence_index, tokens, _, _, ner_tags, _, _, _).


Appending to app.ddlog


In [ ]:
%%file udf/map_person_mention1.py
#!/usr/bin/env python
from deepdive import *

@tsj_extractor
@returns(lambda
        mention_id       = "text",
        mention_text     = "text",
        doc_id           = "text",
        sentence_index   = "int",
        begin_index      = "int",
        end_index        = "int",
        sentence         = "text",
    :[])
def extract(
        doc_id         = "text",
        sentence_index = "int",
        tokens         = "text[]",
        ner_tags       = "text[]",
    ):
    """
    Finds phrases that are continuous words tagged with PERSON.
    """
    num_tokens = len(ner_tags)
    # find all first indexes of series of tokens tagged as PERSON
    first_indexes = (i for i in xrange(num_tokens) if ner_tags[i] == "PERSON" and (i == 0 or ner_tags[i-1] != "PERSON"))
    for begin_index in first_indexes:
        # find the end of the PERSON phrase (consecutive tokens tagged as PERSON)
        end_index = begin_index + 1
        while end_index < num_tokens and ner_tags[end_index] == "PERSON":
            end_index += 1
        end_index -= 1
        # generate a mention identifier
        mention_id = "%s_%d_%d_%d" % (doc_id, sentence_index, begin_index, end_index)
        mention_text = " ".join(map(lambda i: tokens[i], xrange(begin_index, end_index + 1)))
        # Output a tuple for each PERSON phrase
        sentence = " ".join(tokens)
        yield [
            mention_id,
            mention_text,
            doc_id,
            sentence_index,
            begin_index,
            end_index,
            sentence
        ]


Overwriting udf/map_person_mention1.py


In [ ]:
!chmod +x udf/map_person_mention1.py

In [ ]:
%%file -a app.ddlog




Appending to app.ddlog


In [ ]:
!deepdive redo person_mention1

‘run/RUNNING’ -> ‘20211115/152202.564652106’
2021-11-15 15:22:02.874832 process/ext_person_mention1_by_map_person_mention1/run.sh
2021-11-15 15:22:04.789164 deepdive mark 'done' data/person_mention1
‘run/FINISHED’ -> ‘20211115/152202.564652106’


In [ ]:
%%bash
deepdive sql ' select mention_text, sentence from person_mention1 limit 10'

  mention_text   |                                                                                                                           sentence                                                                                                                           
-----------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 John Dawkins    | Degree of pain CBD would like to admit that we might have been a tad unappreciative of just what the Hawke-Keating era minister , John Dawkins , achieved during his all-too brief stint chairing education dunce , Vocation .
 Dawkins         | The risks were all reported in the previous annual report signed by Dawkins , according to his replacement in the chair , Doug Halley .
 Doug Halley     | The risks were all reported in the pre

In [ ]:
import sys
!conda install --yes --prefix {sys.prefix} numpy

In [ ]:
%%bash
deepdive sql 'select  p.mention_text, s.tokens from person_mention p, sentences s where p.doc_id=s.doc_id and p.sentence_index =s.sentence_index  limit 20'
    


In [ ]:
%%file -a app.ddlog

parenthood_candidate(
    p1_id   text,
    p1_name text,
    p2_id   text,
    p2_name text
).


Appending to app.ddlog


In [ ]:
import os

ImportError: No module named 'deepdive'

In [ ]:
%%file -a app.ddlog


parenthood_candidate(p1, p1_name, p2, p2_name) :-
    
    person_mention1(p1, p1_name, same_doc, same_sentence, p1_begin, _,_),
    person_mention1(p2, p2_name, same_doc, same_sentence, p2_begin, _,_),
    p1 < p2,
    p1_name != p2_name,
    p1_begin != p2_begin.

Appending to app.ddlog


In [ ]:
!deepdive redo parenthood_candidate

‘run/RUNNING’ -> ‘20211115/152215.866694513’
2021-11-15 15:22:16.213385 process/ext_parenthood_candidate/run.sh
2021-11-15 15:22:16.469814 deepdive mark 'done' data/parenthood_candidate
‘run/FINISHED’ -> ‘20211115/152215.866694513’


In [ ]:
%%file -a app.ddlog

parenthood_relation2(
    p1_id   text,
    p1_name text,
    p2_id   text,
    p2_name text,
    sentence text
).


Appending to app.ddlog


In [ ]:
%%file -a app.ddlog

function map_parenthood_relation2 over (
        p1_id         text,
        p1_name text,
        p2_id         text,
        p2_name text,
        sentence         text
    ) returns rows like parenthood_relation2
    implementation "udf/map_parenthood_relation2.py" handles tsj lines.

parenthood_relation2 += map_parenthood_relation2(
    p1_id, p1_name, p2_id, p2_name,sentence
) :-
    parenthood_candidate(p1_id,p1_name,p2_id,p2_name),
    person_mention1(p1_id,_,_,_,_,_, sentence).

Appending to app.ddlog


In [ ]:
import sys
print(sys.executable)


/opt/conda/bin/python


In [ ]:
%%file udf/map_parenthood_relation2.py
#!/usr/bin/env python
from deepdive import *
import subprocess
import sys  
import re
#subprocess.check_call([sys.executable, "-m", "pip", "install", "nltk"])
#import nltk
#from nltk.corpus import wordnet
#nltk.download('all',download_dir='/usr/share/nltk_data')
#nltk.data.path.append('/user/local/share/nltk_data')

@tsj_extractor
@returns(lambda
        p1_id       = "text",
        p1_name     = "text",
        p2_id       = "text",
        p2_name   = "int",
        sentence   ="text"
    :[])
def extract(
        p1_id       = "text",
        p1_name     = "text",
        p2_id           = "text",
        p2_name   = "int",
        sentence         = "text",
    ):
    
    parenthoodRegex = re.compile(r'( father | mother | son | doughter )')
    ispa=parenthoodRegex.search(sentence)
    if ispa!=None:
        yield [
        p1_id,
        p1_name,
        p2_id,
        p2_name,
        sentence
        ]
        
    


Overwriting udf/map_parenthood_relation2.py


In [ ]:
import re
sen= "fatheron Ravi Shastri pays homage to BCCIs ` undisputed champion ` Jagmohan Dalmiya Mumbai News.Net - Tuesday 22nd September , 2015 Ravi Shastri on Tuesday paid tributes to Jagmohan Dalmiya , describing the late Board of Control for Cricket in India -LRB- BCCI -RRB- chief as the ` undisputed heavyweight champion ' of the Indian cricket team ... Barkha Singh asks Kejriwal to advise Somnath to cooperate with investigation Mumbai News.Net - Tuesday 22nd September , 2015 Former Delhi Commission for Women -LRB- DCW -RRB- chairperson Barkha Shukla Singh on Tuesday asked Chief Minister Arvind Kejriwal to advise AAP MLA Somnath Bharti to cooperate with the police investigation in ... Delhi Police probing Raghubir Nagar murder case says Bassi Mumbai News.Net - Tuesday 22nd September , 2015 Police Commissioner B.S. Bassi on Tuesday said that investigations are on in the murder case of a woman and her two children in West Delhi 's Raghubir Nagar"
parenthoodRegex = re.compile(r'( father | mother | son | doughter )')
ispa=parenthoodRegex.search(sen)
print(ispa)

None


In [ ]:
!chmod +x udf/map_parenthood_relation2.py

In [ ]:
!deepdive redo parenthood_relation2

‘run/RUNNING’ -> ‘20211028/204157.334992428’
2021-10-28 20:41:57.728039 process/ext_parenthood_relation2_by_map_parenthood_relation2/run.sh
2021-10-28 20:41:59.235508 deepdive mark 'done' data/parenthood_relation2
‘run/FINISHED’ -> ‘20211028/204157.334992428’


In [ ]:
%%bash
deepdive sql 'select p1_name, p2_name, sentence from parenthood_relation2 limit 10'

         p1_name          |     p2_name     |                                                                                                           sentence                                                                                                           
--------------------------+-----------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Jake                     | Jesy Nelson     | The news comes just weeks after Coleen 's younger son , Jake , announced his engagement to Little Mix star , Jesy Nelson .
 Jake                     | Coleen          | The news comes just weeks after Coleen 's younger son , Jake , announced his engagement to Little Mix star , Jesy Nelson .
 Dell Curry               | Seth Curry      | Dell Curry is many things '' '' the Charlotte Hornets '' ™ all-time leading scor

In [ ]:
%%bash
deepdive query '
    name1, name2, doc, sentence | 20
    ?- parenthood_candidate(p1, name1, p2, name2),
       person_mention(p1, _, doc, sentence, _, _).
'

      name1       |      name2       |                 doc                  | sentence 
------------------+------------------+--------------------------------------+----------
 Jonathan Bloom   | Patrick Page     | 8b31ede3-0f3b-431a-86a3-342ee18cfd83 |       11
 Bloom            | Doherty          | 8b31ede3-0f3b-431a-86a3-342ee18cfd83 |       13
 Sherlock Holmes  | Sherlock         | 8b31ede3-0f3b-431a-86a3-342ee18cfd83 |       17
 Kennedy          | Ted Kennedy      | 0d07e617-00d4-4866-aee2-0ae197ae366f |       17
 Alex             | Liz              | 0d07e617-00d4-4866-aee2-0ae197ae366f |       19
 Jay Scott        | Liz              | 0d07e617-00d4-4866-aee2-0ae197ae366f |       19
 Jay Scott        | Alex             | 0d07e617-00d4-4866-aee2-0ae197ae366f |       19
 Dame Joan        | Jackie           | df13cc43-53fd-4f09-9a7e-d69b12a4adc0 |       13
 Peter Holm       | Jackie           | df13cc43-53fd-4f09-9a7e-d69b12a4adc0 |       14
 Dame Joan        | Jackie           | df

### 1.4. Extracting features for each candidate
Finally, we will extract a set of **features** for each candidate:

In [ ]:
%%file -a app.ddlog

## Feature Extraction #########################################################
 
# Feature extraction (using DDLIB via a UDF) at the relation level
parenthood_feature(
    p1_id   text,
    p2_id   text,
    feature text
).


Appending to app.ddlog


In [ ]:
%%file udf/extract_parenthood_features.py
#!/usr/bin/env python
from deepdive import *
import ddlib

@tsj_extractor
@returns(lambda
        p1_id   = "text",
        p2_id   = "text",
        feature = "text",
    :[])
def extract(
        p1_id          = "text",
        p2_id          = "text",
        p1_begin_index = "int",
        p1_end_index   = "int",
        p2_begin_index = "int",
        p2_end_index   = "int",
        doc_id         = "text",
        sent_index     = "int",
        tokens         = "text[]",
        lemmas         = "text[]",
        pos_tags       = "text[]",
        ner_tags       = "text[]",
        dep_types      = "text[]",
        dep_parents    = "int[]",
    ):
    """
    Uses DDLIB to generate features for the spouse relation.
    """
    # Create a DDLIB sentence object, which is just a list of DDLIB Word objects
    sent = []
    for i,t in enumerate(tokens):
        sent.append(ddlib.Word(
            begin_char_offset=None,
            end_char_offset=None,
            word=t,
            lemma=lemmas[i],
            pos=pos_tags[i],
            ner=ner_tags[i],
            dep_par=dep_parents[i] - 1,  # Note that as stored from CoreNLP 0 is ROOT, but for DDLIB -1 is ROOT
            dep_label=dep_types[i]))

    # Create DDLIB Spans for the two person mentions
    p1_span = ddlib.Span(begin_word_id=p1_begin_index, length=(p1_end_index-p1_begin_index+1))
    p2_span = ddlib.Span(begin_word_id=p2_begin_index, length=(p2_end_index-p2_begin_index+1))

    # Generate the generic features using DDLIB
    for feature in ddlib.get_generic_features_relation(sent, p1_span, p2_span):
        yield [p1_id, p2_id, feature]


Writing udf/extract_parenthood_features.py


In [ ]:
!chmod +x udf/extract_parenthood_features.py

In [ ]:
%%file -a app.ddlog

function extract_parenthood_features over (
        p1_id          text,
        p2_id          text,
        p1_begin_index int,
        p1_end_index   int,
        p2_begin_index int,
        p2_end_index   int,
        doc_id         text,
        sent_index     int,
        tokens         text[],
        lemmas         text[],
        pos_tags       text[],
        ner_tags       text[],
        dep_types      text[],
        dep_tokens     int[]
    ) returns rows like spouse_feature
    implementation "udf/extract_parenthood_features.py" handles tsj lines.

parenthood_feature += extract_parenthood_features(
    p1_id, p2_id, p1_begin_index, p1_end_index, p2_begin_index, p2_end_index,
    doc_id, sent_index, tokens, lemmas, pos_tags, ner_tags, dep_types, dep_tokens
) :-
    person_mention1(p1_id, _, doc_id, sent_index, p1_begin_index, p1_end_index,_),
    person_mention1(p2_id, _, doc_id, sent_index, p2_begin_index, p2_end_index,_),
    sentences(doc_id, sent_index, tokens, lemmas, pos_tags, ner_tags, _, dep_types, dep_tokens).

Appending to app.ddlog


In [ ]:
%%file -a app.ddlog

# assign the resolved labels for the spouse relation
has_parenthood(p1_id, p2_id) =:- spouse_label_resolved(p1_id, p2_id, l).

In [ ]:
!deepdive redo parenthood_feature

app.ddlog: updated since last `deepdive compile`
‘run/compiled’ -> ‘20211029/164052.995395286’
‘run/RUNNING’ -> ‘20211029/164055.155625819’
2021-10-29 16:40:55.487055 process/ext_parenthood_feature_by_extract_parenthood_features/run.sh
2021-10-29 16:41:04.460449 deepdive mark 'done' data/parenthood_feature
‘run/FINISHED’ -> ‘20211029/164055.155625819’


In [ ]:
!deepdive query '| 20 ?- parenthood_feature(_, _, feature).'

Distant supervision with data and rules

In [ ]:
%%file -a app.ddlog

# distant supervision using data from DBpedia

parenthood_dbpedia_data(
    url text,
    name text,
    parent text
).

Appending to app.ddlog


In [ ]:
!deepdive redo parenthood_dbpedia_data

app.ddlog: updated since last `deepdive compile`
‘run/compiled’ -> ‘20211124/184911.246363438’
‘run/RUNNING’ -> ‘20211124/184912.554342710’
2021-11-24 18:49:12.778655 process/init/relation/parenthood_dbpedia_data/run.sh
2021-11-24 18:49:15.818877 deepdive mark 'done' data/parenthood_dbpedia_data
‘run/FINISHED’ -> ‘20211124/184912.554342710’


In [ ]:
!deepdive query '| 20 ?- parenthood_dbpedia_data(url,name, parent).'

                           url                           |            name            | parent 
---------------------------------------------------------+----------------------------+--------
 http://dbpedia.org/resource/Mike_Richardson_(publisher) | Mike Richardson            | NULL
 http://dbpedia.org/resource/Minami_Ozaki                | Minami Ozaki               | NULL
 http://dbpedia.org/resource/Hiroya_Oku                  | Hiroya Oku                 | NULL
 http://dbpedia.org/resource/Kazimieras_G._Prapuolenis   | Kaz                        | NULL
 http://dbpedia.org/resource/Kelley_Puckett              | Connor Puckett             | NULL
 http://dbpedia.org/resource/Alex_Anderson_(cartoonist)  | Alex Anderson              | NULL
 http://dbpedia.org/resource/Izumi_Aso                   | {Izumi Aso|麻生 いずみ}    | NULL
 http://dbpedia.org/resource/Goseki_Kojima               | {Goseki Kojima|小島 剛夕}  | NULL
 http://dbpedia.org/resource/Ho_Che_Anderson             | Ho Che Anderso

In [ ]:
!deepdive  sql 'SELECT t1.name, t2.name as parent FROM parenthood_dbpedia_data  t1 JOIN  parenthood_dbpedia_data  t2 ON t1.parent = t2.url limit 10'


                                         name                                          |                                                                         parent                                                                          
---------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------
 Ladahachandra                                                                         | Kalyanachandra
 {Dennis Rigg|Mark Hughes}                                                             | Kate Sugden
 {Alex Kinski|Bobby Hoyland|Gabrielle Walker|Genevieve Doyle|Kayla Thomas|Kim Timmins} | {Anne Baxter|Cameron Robinson|Charlie Hoyland|Christine Rodd|Glenn Forrest|Guy Sykes|Kerry Mangel|Mishka Schneiderova|Robert Robinson|Teresa Cammeniti}
 Tim Daly                                                             

In [ ]:
%%file -a app.ddlog

# distant supervision using data from DBpedia

parenthood_dbpedia(
    name text,
    parent text
).

Appending to app.ddlog


In [ ]:
%%file -a app.ddlog


parenthood_dbpedia(p1_name,p2_name) :-
    
    parenthood_dbpedia_data(url1,p1_name, same ),
    parenthood_dbpedia_data(same,p2_name, parent2).
    

Appending to app.ddlog


In [ ]:
!deepdive redo parenthood_dbpedia

app.ddlog: updated since last `deepdive compile`
‘run/compiled’ -> ‘20211124/193241.568933185’
‘run/RUNNING’ -> ‘20211124/193242.863499150’
2021-11-24 19:32:43.140124 process/ext_parenthood_dbpedia/run.sh
2021-11-24 19:32:43.422172 deepdive mark 'done' data/parenthood_dbpedia
‘run/FINISHED’ -> ‘20211124/193242.863499150’


In [ ]:
!deepdive query '| 20 ?- parenthood_dbpedia(name, parent).'

                           name                            |                                                         parent                                                          
-----------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------
 Guy I of Luxembourg-Ligny                                 | John I of Luxembourg Lord of Ligny
 Dan I                                                     | Radu I
 {Mathu Shah|Min Dawlya}                                   | {Ba Saw Phyu|Kalima Shah}
 {Youlan|幼蘭}                                             | {Guwalgiya Ronglu|瓜爾佳·榮祿}
 Min Sithu                                                 | Sithu Kyawhtin of Toungoo
 {Shō Ken|尚賢王}                                          | {Shō Hō|尚豊王}
 {Prince Azam Jah|Sahebzada Mir Himayat Ali Khan Azam Jah} | Mir Osman Ali Khan Chin Chilich Khan Nizam-Ul-Mulk Asaf Jah VII
 Sanda Wuduroma o

In [ ]:
%%file -a app.ddlog

## Distant Supervision ########################################################
parenthood_label(
    p1_id   text,
    p2_id   text,
    label   int,
    rule_id text
).


Appending to app.ddlog


In [ ]:
%%file -a app.ddlog

# make sure all pairs in spouse_candidate are considered as unsupervised examples
parenthood_label(p1,p2, 0, NULL) :-
    parenthood_candidate(p1, _, p2, _).

Appending to app.ddlog


Supervising parenthood candidates with DBpedia data

In [ ]:
%%file -a app.ddlog

parenthood_label(p1,p2, 1, "from_dbpedia") :-
    parenthood_candidate(p1, p1_name, p2, p2_name),
    parenthood_dbpedia(n1, n2),
    [ lower(n1) = lower(p1_name), lower(n2) = lower(p2_name) ;
      lower(n2) = lower(p1_name), lower(n1) = lower(p2_name) ].

Appending to app.ddlog


Using heuristic rules for distant supervision

In [ ]:
%%file -a app.ddlog

# supervision by heuristic rules in a UDF
function supervise over (
        p1_id text, p1_begin int, p1_end int,
        p2_id text, p2_begin int, p2_end int,
        doc_id         text,
        sentence_index int,
        sentence_text  text,
        tokens         text[],
        lemmas         text[],
        pos_tags       text[],
        ner_tags       text[],
        dep_types      text[],
        dep_tokens     int[]
    ) returns (
        p1_id text, p2_id text, label int, rule_id text
    )
    implementation "udf/supervise_parenthood.py" handles tsj lines.

parenthood_label += supervise(
    p1_id, p1_begin, p1_end,
    p2_id, p2_begin, p2_end,
    doc_id, sentence_index,
    tokens, lemmas, pos_tags, ner_tags, dep_types, dep_token_indexes
) :-
    parenthood_candidate(p1_id, _, p2_id, _),
    person_mention1(p1_id, p1_text, doc_id, sentence_index, p1_begin, p1_end,_),
    person_mention1(p2_id, p2_text,      _,              _, p2_begin, p2_end,_),
    sentences(
        doc_id, sentence_index,
        tokens, lemmas, pos_tags, ner_tags, _, dep_types, dep_token_indexes
    ).

Appending to app.ddlog


In [ ]:
%%file udf/supervise_parenthood.py
#!/usr/bin/env python
from deepdive import *
import random
from collections import namedtuple

ParenthoodLabel = namedtuple('ParenthoodLabel', 'p1_id, p2_id, label, type')

@tsj_extractor
@returns(lambda
        p1_id   = "text",
        p2_id   = "text",
        label   = "int",
        rule_id = "text",
    :[])
# heuristic rules for finding positive/negative examples of parenthood relationship mentions
def supervise(
        p1_id="text", p1_begin="int", p1_end="int",
        p2_id="text", p2_begin="int", p2_end="int",
        doc_id="text", sentence_index="int",
        tokens="text[]", lemmas="text[]", pos_tags="text[]", ner_tags="text[]",
        dep_types="text[]", dep_token_indexes="int[]",
    ):

    # Constants
    parent = frozenset(["father", "mother" "doughter","son"])
    FAMILY = frozenset(["mother", "father", "sister", "brother", "brother-in-law"])
    MAX_DIST = 10

    # Common data objects
    p1_end_idx = min(p1_end, p2_end)
    p2_start_idx = max(p1_begin, p2_begin)
    p2_end_idx = max(p1_end,p2_end)
    intermediate_lemmas = lemmas[p1_end_idx+1:p2_start_idx]
    intermediate_ner_tags = ner_tags[p1_end_idx+1:p2_start_idx]
    tail_lemmas = lemmas[p2_end_idx+1:]
    parenthood = ParenthoodLabel(p1_id=p1_id, p2_id=p2_id, label=None, type=None)

    # Rule: Candidates that are too far apart
    if len(intermediate_lemmas) > MAX_DIST:
        yield parenthood._replace(label=-1, type='neg:far_apart')

    # Rule: Candidates that have a third person in between
    if 'PERSON' in intermediate_ner_tags:
        yield parenthood._replace(label=-1, type='neg:third_person_between')

    # Rule: Sentences that contain wife/husband in between
    #         (<P1>)([ A-Za-z]+)(wife|husband)([ A-Za-z]+)(<P2>)
    if len(parent.intersection(intermediate_lemmas)) > 0:
        yield parenthood._replace(label=1, type='pos:father_mother__son_doughter_between')

    # Rule: Sentences that contain familial relations:
    #         (<P1>)([ A-Za-z]+)(brother|stster|father|mother)([ A-Za-z]+)(<P2>)
    if len(FAMILY.intersection(intermediate_lemmas)) > 0:
        yield parenthood._replace(label=-1, type='neg:familial_between')


Overwriting udf/supervise_parenthood.py


In [ ]:
!chmod +x udf/supervise_parenthood.py

Resolving multiple labels per example with majority vote

In [ ]:
%%file -a app.ddlog

# resolve multiple labels by majority vote (summing the labels in {-1,0,1})
parenthood_label_resolved(p1_id, p2_id, SUM(vote)) :-
    parenthood_label(p1_id, p2_id, vote, rule_id).

Appending to app.ddlog


In [ ]:
%%file -a app.ddlog

# assign the resolved labels for the parenthood relation
has_parenthood(p1_id, p2_id) = if l > 0 then TRUE
                      else if l < 0 then FALSE
                      else NULL end :- parenthood_label_resolved(p1_id, p2_id, l).

Appending to app.ddlog


In [ ]:
!deepdive redo has_parenthood

app.ddlog: updated since last `deepdive compile`
2021-11-24 18:55:28.167823 ‘run/LATEST.COMPILE’ -> ‘20211124/185528.130952660’
2021-11-24 18:55:28.167897 ‘run/RUNNING.COMPILE’ -> ‘20211124/185528.130952660’
2021-11-24 18:55:28.167917 Parsing DeepDive application (/ConfinedWater/deepdive-examples/spouse) to generate:
2021-11-24 18:55:28.167932  run/compiled/deepdive.conf
2021-11-24 18:55:28.167950   from app.ddlog
2021-11-24 18:55:28.608026 [error] app.ddlog[168.13] failure: `)' expected but `t' found
2021-11-24 18:55:28.608112 
2021-11-24 18:55:28.608138     p1_id   text,
2021-11-24 18:55:28.608164 
2021-11-24 18:55:28.608202             ^
2021-11-24 18:55:28.634243 ‘run/ABORTED.COMPILE’ -> ‘20211124/185528.130952660’


In [ ]:
%%file -a app.ddlog

## Inference Rules ############################################################
 
# Features
@weight(f)
has_parenthood(p1_id, p2_id) :-
    parenthood_feature(p1_id, p2_id, f).

Appending to app.ddlog


In [ ]:
!deepdive redo has_parenthood

‘run/RUNNING’ -> ‘20211130/150109.074664028’
2021-11-30 15:01:09.653519 process/ext_parenthood_label__0_by_supervise/run.sh
2021-11-30 15:01:11.348928 deepdive mark 'done' data/parenthood_label__0
2021-11-30 15:01:11.398855 process/ext_parenthood_label/run.sh
2021-11-30 15:01:11.690661 deepdive mark 'done' data/parenthood_label
2021-11-30 15:01:11.745179 process/ext_parenthood_label_resolved/run.sh
2021-11-30 15:01:11.977631 deepdive mark 'done' data/parenthood_label_resolved
2021-11-30 15:01:12.032495 process/ext_has_parenthood/run.sh
2021-11-30 15:01:12.360874 deepdive mark 'done' data/has_parenthood
‘run/FINISHED’ -> ‘20211130/150109.074664028’


Now, we can take a brief look at how many candidates are supervised by different rules, which will look something like the table below. Obviously, the counts will vary depending on your input corpus.

In [ ]:
!deepdive query 'rule, @order_by COUNT(1) ?- parenthood_label(p1,p2, label, rule).'

                  rule                   | COUNT(1) 
-----------------------------------------+----------
 pos:father_mother__son_doughter_between |       23
 neg:familial_between                    |       44
 neg:third_person_between                |      454
 neg:far_apart                           |      485
                                         |     1017
(5 rows)



Learning and inference: model specification

Specifying features

In [ ]:
%%file -a app.ddlog

## Inference Rules ############################################################
 
# Features
@weight(f)
has_parenthood(p1_id, p2_id) :-
    parenthood_feature(p1_id, p2_id, f).

Appending to app.ddlog


Specifying connections between variables

In [ ]:
!deepdive redo probabilities

app.ddlog: updated since last `deepdive compile`
‘run/compiled’ -> ‘20211130/150754.555185240’
‘run/RUNNING’ -> ‘20211130/150756.223088165’
2021-11-30 15:07:57.138265 process/ext_parenthood_feature_by_extract_parenthood_features/run.sh
2021-11-30 15:08:04.924395 deepdive mark 'done' data/parenthood_feature
2021-11-30 15:08:04.978230 process/grounding/from_grounding/run.sh
2021-11-30 15:08:05.030924 process/grounding/variable/has_parenthood/materialize/run.sh
2021-11-30 15:08:15.941028 process/grounding/variable_assign_id/run.sh
2021-11-30 15:08:16.509385 process/grounding/factor/inf_istrue_has_parenthood/materialize/run.sh
2021-11-30 15:08:28.243498 process/grounding/assign_weight_id/run.sh
2021-11-30 15:08:28.893147 process/grounding/factor/inf_istrue_has_parenthood/0/dump/run.sh
2021-11-30 15:08:29.371694 process/grounding/factor/inf_istrue_has_parenthood/dump_weights/run.sh
2021-11-30 15:08:29.673472 process/grounding/global_weight_table/run.sh
2021-11-30 15:08:30.318656 process/gro

In [ ]:
!deepdive sql 'SELECT p1_id, p2_id, expectation FROM has_parenthood_inference ORDER BY random() LIMIT 50'

                     p1_id                      |                     p2_id                      | expectation 
------------------------------------------------+------------------------------------------------+-------------
 0d07e617-00d4-4866-aee2-0ae197ae366f_19_16_16  | 0d07e617-00d4-4866-aee2-0ae197ae366f_19_9_9    |       0.003
 c5f8a528-cc0f-4f3e-aaef-b9e3b6b00325_4_16_17   | c5f8a528-cc0f-4f3e-aaef-b9e3b6b00325_4_2_3     |       0.003
 9662058b-fca5-4771-8058-c7fd7bd548a3_84_0_0    | 9662058b-fca5-4771-8058-c7fd7bd548a3_84_2_2    |       0.003
 df13cc43-53fd-4f09-9a7e-d69b12a4adc0_2_0_0     | df13cc43-53fd-4f09-9a7e-d69b12a4adc0_2_14_18   |       0.008
 f18bbd65-57a4-414a-b78b-356b9ecb65f0_16_1_1    | f18bbd65-57a4-414a-b78b-356b9ecb65f0_16_8_8    |       0.018
 e505b558-5db9-4d92-aec0-3b32341499b8_5_229_229 | e505b558-5db9-4d92-aec0-3b32341499b8_5_30_31   |           0
 43c02bac-d556-4851-9b04-7773700759b6_23_35_36  | 43c02bac-d556-4851-9b04-7773700759b6_23_48_48  |       0.003

Error analysis and debugging

In [ ]:
!deepdive do calibration-plots

‘run/RUNNING’ -> ‘20211130/151120.378767294’
2021-11-30 15:11:21.153645 process/model/calibration/run.sh
‘run/FINISHED’ -> ‘20211130/151120.378767294’


In [ ]:
!mindbender search drop; mindbender search update

In [ ]:
!mindbender search gui

Launching Elasticsearch for http://localhost:9200 from /ConfinedWater/deepdive-examples/spouse/search
30 Nov 15:45:26 - Loaded 0 Mindtagger tasks: 
30 Nov 15:45:26 - Mindbender GUI started at http://42cf3f8c4652:8000/
